In [1]:
import random
import numpy as np
import tensorflow as tf

random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [2]:
from pathlib import Path
from keras import Sequential
from keras.layers import *
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping
from sklearn.naive_bayes import GaussianNB

from src.features.encoder import FeatureEncoder
from src.data.data_loader import load_dataset, Species
from src.models.reporting.model_report import ModelReport
from src.models.reporting.single_report import generate_report

In [5]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

### Human Training & Testing

In [3]:
human_x_test, human_y_test = load_dataset(Species.human, independent=True,
                                          encoding=lambda x: FeatureEncoder.ps(x, 3))
human_x_train, human_y_train = load_dataset(Species.human, independent=False,
                                            encoding=lambda x: FeatureEncoder.ps(x, 3))

In [11]:
human_model = Sequential([
    Dense(1216, input_dim=1216, activation='relu'),
    Dropout(0.8),
    Dense(500, activation='relu'),
    Dropout(0.8),
    Dense(1, activation='sigmoid'),
])

human_model.compile(
    loss=BinaryCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'],
)

human_model.fit(human_x_train, human_y_train, epochs=40, callbacks=[early_stopping],
                validation_data=(human_x_test, human_y_test))

Epoch 1/40
31/31 [==============================] - 0s 6ms/step - loss: 0.7215 - accuracy: 0.4879 - val_loss: 0.6851 - val_accuracy: 0.5100
Epoch 2/40
31/31 [==============================] - 0s 5ms/step - loss: 0.6701 - accuracy: 0.5929 - val_loss: 0.6570 - val_accuracy: 0.6550
Epoch 3/40
31/31 [==============================] - 0s 4ms/step - loss: 0.5985 - accuracy: 0.6717 - val_loss: 0.6201 - val_accuracy: 0.6800
Epoch 4/40
31/31 [==============================] - 0s 5ms/step - loss: 0.5133 - accuracy: 0.7515 - val_loss: 0.6145 - val_accuracy: 0.6950
Epoch 5/40
31/31 [==============================] - 0s 4ms/step - loss: 0.4046 - accuracy: 0.8273 - val_loss: 0.6565 - val_accuracy: 0.6500
Epoch 6/40
31/31 [==============================] - 0s 4ms/step - loss: 0.3256 - accuracy: 0.8556 - val_loss: 0.6995 - val_accuracy: 0.6750
Epoch 7/40
31/31 [==============================] - 0s 4ms/step - loss: 0.2571 - accuracy: 0.9020 - val_loss: 0.7821 - val_accuracy: 0.6300
Epoch 8/40
31/31 [==

Epoch 1/40
31/31 [==============================] - 1s 8ms/step - loss: 0.7013 - accuracy: 0.5333 - val_loss: 0.6699 - val_accuracy: 0.6500
Epoch 2/40
31/31 [==============================] - 0s 3ms/step - loss: 0.6376 - accuracy: 0.6212 - val_loss: 0.6435 - val_accuracy: 0.6850
Epoch 3/40
31/31 [==============================] - 0s 4ms/step - loss: 0.5785 - accuracy: 0.6626 - val_loss: 0.6241 - val_accuracy: 0.6500
Epoch 4/40
31/31 [==============================] - 0s 4ms/step - loss: 0.5023 - accuracy: 0.7313 - val_loss: 0.6099 - val_accuracy: 0.6800
Epoch 5/40
31/31 [==============================] - 0s 4ms/step - loss: 0.3907 - accuracy: 0.8242 - val_loss: 0.6436 - val_accuracy: 0.6550
Epoch 6/40
31/31 [==============================] - 0s 4ms/step - loss: 0.2855 - accuracy: 0.8808 - val_loss: 0.7353 - val_accuracy: 0.6400
Epoch 7/40
31/31 [==============================] - 0s 4ms/step - loss: 0.1930 - accuracy: 0.9384 - val_loss: 0.8466 - val_accuracy: 0.6600
Epoch 8/40
31/31 [==

### Yeast Training & Testing

In [102]:
yeast_x_test, yeast_y_test = load_dataset(Species.yeast, independent=True,
                                          encoding=lambda x: FeatureEncoder.pse_knc(x, info, 3, 2, 0.1))
yeast_x_train, yeast_y_train = load_dataset(Species.yeast, independent=False,
                                            encoding=lambda x: FeatureEncoder.pse_knc(x, info, 3, 2, 0.1))

In [105]:
yeast_model = Sequential([
    Dense(66, input_dim=66, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.8),
    Dense(1, activation='sigmoid'),
])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

yeast_model.compile(loss=BinaryCrossentropy(from_logits=False), optimizer='adam', metrics=['accuracy'])

In [106]:
yeast_model.fit(yeast_x_train, yeast_y_train, epochs=40, callbacks=[early_stopping],
                validation_data=(yeast_x_test, yeast_y_test))

Epoch 1/40
20/20 [==============================] - 0s 4ms/step - loss: 0.6965 - accuracy: 0.4920 - val_loss: 0.6933 - val_accuracy: 0.5050
Epoch 2/40
20/20 [==============================] - 0s 1ms/step - loss: 0.6934 - accuracy: 0.4968 - val_loss: 0.6931 - val_accuracy: 0.4900
Epoch 3/40
20/20 [==============================] - 0s 1ms/step - loss: 0.6926 - accuracy: 0.5223 - val_loss: 0.6928 - val_accuracy: 0.5100
Epoch 4/40
20/20 [==============================] - 0s 1ms/step - loss: 0.6924 - accuracy: 0.5318 - val_loss: 0.6922 - val_accuracy: 0.5300
Epoch 5/40
20/20 [==============================] - 0s 1ms/step - loss: 0.6905 - accuracy: 0.5446 - val_loss: 0.6916 - val_accuracy: 0.5450
Epoch 6/40
20/20 [==============================] - 0s 1ms/step - loss: 0.6935 - accuracy: 0.4968 - val_loss: 0.6917 - val_accuracy: 0.6150
Epoch 7/40
20/20 [==============================] - 0s 1ms/step - loss: 0.6872 - accuracy: 0.5669 - val_loss: 0.6916 - val_accuracy: 0.5900
Epoch 8/40
20/20 [==

In [107]:
human_model_report = ModelReport.generate(human_model, human_x_test, human_y_test, is_keras=True)
yeast_model_report = ModelReport.generate(yeast_model, yeast_x_test, yeast_y_test, is_keras=True)

7/7 [==============================] - 0s 451us/step


In [108]:
generate_report({Species.human.value: human_model_report, Species.yeast.value: yeast_model_report}, Path('nn/pseknc'))

note: Running TeX ...
note: Rerunning TeX because "report.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `nn/pseknc/report.pdf` (35.46 KiB)
note: Skipped writing 1 intermediate files (use --keep-intermediates to keep them)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [109]:
def select_features(features):
    selection = [4, 21, 20, 0, 5, 65, 64, 1, 17, 16, 60, 15, 25, 54, 3, 11, 46,
                 41, 40, 39, 7, 2, 42, 36, 31, 12, 24, 35, 18, 62, 61, 53, 8, 50,
                 56, 43, 19, 28, 9, 59, 37, 38, 33, 48, 44, 45, 63, 10, 22, 27,
                 49, 57, 55, 14, 51, 47, 52, 26, 23, 29, 30, 13, 6, 58]
    imp_features = list(map(lambda x: f'pse_knc_{x + 1}', selection))
    return features[imp_features]

In [110]:
human_classifier = GaussianNB()

human_classifier.fit(select_features(human_x_train), human_y_train)
human_classifier.score(select_features(human_x_test), human_y_test)

0.585

In [111]:
yeast_classifier = GaussianNB()

yeast_classifier.fit(select_features(yeast_x_train), yeast_y_train)
yeast_classifier.score(select_features(yeast_x_test), yeast_y_test)

0.52

In [113]:
human_model_report = ModelReport.generate(human_classifier, select_features(human_x_test), human_y_test)
yeast_model_report = ModelReport.generate(yeast_classifier, select_features(yeast_x_test), yeast_y_test)

In [114]:
generate_report({Species.human.value: human_model_report, Species.yeast.value: yeast_model_report}, Path('ml/pseknc'))

note: Running TeX ...
note: Rerunning TeX because "report.aux" changed ...
note: Running xdvipdfmx ...
note: Writing `ml/pseknc/report.pdf` (35.29 KiB)
note: Skipped writing 1 intermediate files (use --keep-intermediates to keep them)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>